In [1]:
# install.packages('dplyr',repos="http://cran.us.r-project.org/")

In [2]:
library(RSQLite)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
con <- dbConnect(SQLite(), dbname="/Users/Amardeep/Documents/Semester_2/Dic/lab_2/database.sqlite")

Please change the connection Url accordingly

## Making the ideal Player data frame

In [4]:
Player1 <- dbGetQuery(con, "SELECT * FROM `Player`")
player_name = data.frame(Player1$player_api_id,Player1$player_name)
colnames(player_name) <- c("player_api_id", "player_name")

The above block of code creates two data frames Player1 (contains all coloumns from table, Player) and player_name(contains the player id and player name).

In [5]:
Match <- dbGetQuery(con, "SELECT * FROM `Match`")

above block of code creates a Data frame called Match having all the attributes from Match table

In [6]:
appearences = c(Match$home_player_1, Match$home_player_2, Match$home_player_3, Match$home_player_4, Match$home_player_5, Match$home_player_6, Match$home_player_7, Match$home_player_8, Match$home_player_9, Match$home_player_10, Match$home_player_11, Match$away_player_1, Match$away_player_2, Match$away_player_3, Match$away_player_4, Match$away_player_5, Match$away_player_6, Match$away_player_7, Match$away_player_8, Match$away_player_9, Match$away_player_10, Match$away_player_11)

In [7]:
appearences <- as.data.frame(appearences)
nrow(appearences)

[1] 571538

The above blocks of code concatinate all the home player and away player fields into one and create a data frame

In [8]:
appearences <- appearences %>% filter(!is.na(appearences)) %>% arrange(appearences)

In [9]:
no_appear <- appearences %>% count(appearences, sort = TRUE)

colnames(no_appear) <- c("player_api_id", "total_appearances")
print(no_appear)

# A tibble: 11,060 × 2
   player_api_id total_appearances
           <int>             <int>
1          31293               300
2          41097               294
3          33764               286
4          42422               286
5          26295               282
6          31465               282
7          31432               275
8          26168               268
9          26399               262
10         94550               260
# ... with 11,050 more rows


The code block above counts the repeated entries in the coloumn and creates a new row containing frequency of the player api id.

In [10]:
Player2 <- dbGetQuery(con, "SELECT * FROM `Player_Attributes`")
#Creates a dataframe containing all the player attributes.

In [11]:
player_att <- mutate(Player2, gk_rating = gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes)
#Adds all goal keeper attributes in a single coloumn called gk_rating

gk_rating <- data.frame(player_att$player_api_id,player_att$gk_rating)
colnames(gk_rating) <- c("player_api_id", "gk_rating")
gk_rating <- gk_rating %>% arrange(desc(gk_rating))
# creates a new data frame called gk_attributes containing only the player api id and gk_attributes

gk_rating <- gk_rating %>% group_by(player_api_id) %>% summarise(gk_rating=max(gk_rating))
# groups similar player ids together and finds the maximum gk_attibute from the group

gk_rating <- gk_rating %>% arrange(desc(gk_rating))
# arranges the data frame in decreasing order of gk_ratings

head(gk_rating)

player_api_id,gk_rating
30717,449
39989,447
30859,445
30657,442
27299,440
30989,438


In [12]:
ovr_rating <- data.frame(player_att$player_api_id,player_att$overall_rating)
colnames(ovr_rating) <- c("player_api_id", "overall_rating")
# creates a new data frame called ovr_rating containing only the player api id and overall attributes

ovr_rating <- ovr_rating %>% group_by(player_api_id) %>% summarise(overall_rating=max(overall_rating))
# groups similar player ids together and finds the maximum overall_rating from the group

ovr_rating <- ovr_rating %>% arrange(desc(overall_rating))
# arranges the data frame in decreasing order

head(ovr_rating)

player_api_id,overall_rating
30981,94
30717,93
30829,93
30893,93
39854,92
39989,92


In [13]:
l_id <- data.frame(Match$home_player_1,Match$league_id)
colnames(l_id) <- c("player_api_id", "league_id")

for(i in 2:11){
    l_temp <- data.frame(Match[[paste("home_player_", i, sep="")]],Match$league_id)
    colnames(l_temp) <- c("player_api_id", "league_id")
    l_id <- rbind(l_id,l_temp)
}

for(i in 1:11){
    l_temp <- data.frame(Match[[paste("away_player_", i, sep="")]],Match$league_id)
    colnames(l_temp) <- c("player_api_id", "league_id")
    l_id <- rbind(l_id,l_temp)
}
# loops to add all the home team api ids and their respective league ids to two coloumns

l_id <- l_id %>% filter(!is.na(player_api_id)) %>% arrange(player_api_id)
l_id <- l_id %>% group_by(player_api_id) %>% distinct(league_id)  %>% count(player_api_id, sort = TRUE)
# Groups the players according to player api ids, finds the distinct entries and calculates its frequency

colnames(l_id) <- c("player_api_id", "league_appearances")

nrow(l_id)
head(l_id)

[1] 11060

player_api_id,league_appearances
42119,6
116750,5
178538,5
22543,4
24123,4
25816,4


Now that we have all individual data frames having the attributes that we want in the final table, we join all these data frames together using left join.

In [14]:
Player <- player_name %>% left_join(no_appear, by = "player_api_id")
Player <- Player %>% left_join(gk_rating, by = "player_api_id")
Player <- Player %>% left_join(ovr_rating, by = "player_api_id")
Player <- Player %>% left_join(l_id, by = "player_api_id")
# Player <- Player %>% arrange(desc(overall_rating))

#left join to add all the data frenes based on Player api ids.
nrow(Player)
head(Player)

[1] 11060

player_api_id,player_name,total_appearances,gk_rating,overall_rating,league_appearances
505942,Aaron Appindangoye,8,43,67,1
155782,Aaron Cresswell,75,121,74,1
162549,Aaron Doran,104,125,71,1
30572,Aaron Galindo,10,111,75,1
23780,Aaron Hughes,162,129,78,1
27316,Aaron Hunt,158,132,79,1


## Making the ideal Team data frame

In [26]:
team1 <- dbGetQuery(con, "SELECT * FROM `Team`")
team_att <- dbGetQuery(con, "SELECT * FROM `Team_Attributes`")
nrow(team_att)

[1] 1458

In [16]:
Team_name <- data.frame(team1$team_api_id, team1$team_long_name, team1$team_short_name)
colnames(Team_name) <- c("team_api_id", "team_long_name", "team_short_name")
head(Team_name)

team_api_id,team_long_name,team_short_name
9987,KRC Genk,GEN
9993,Beerschot AC,BAC
10000,SV Zulte-Waregem,ZUL
9994,Sporting Lokeren,LOK
9984,KSV Cercle Brugge,CEB
8635,RSC Anderlecht,AND


Creates a data frame having the team names and team ids.

In [46]:
buildup1 <- data.frame(team_att$team_api_id, team_att$buildUpPlaySpeed, team_att$buildUpPlayDribbling, team_att$buildUpPlayPassing)

colnames(buildup1) <- c("team_api_id", "buildUpPlaySpeed", "buildUpPlayDribbling", "buildUpPlayPassing")
 buildup1[is.na(buildup1)] <- 0
# transforms all NA to 0
buildup1 <- mutate(buildup1, buildup_rating = buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing)
# adds all build up attributes together

buildup1 <- data.frame(buildup1$team_api_id, buildup1$buildup_rating)
colnames(buildup1) <- c("team_api_id", "buildUp_rating")
#creates a new data frame having only the team api id and the build up rating

buildup1 <- buildup1 %>% group_by(team_api_id) %>% summarise(buildUp_rating=max(buildUp_rating))
# groups similar player ids together and finds the maximum build up rating from the group


nrow(buildup1)
head(buildup1)

[1] 288

team_api_id,buildUp_rating
1601,139
1773,153
1957,158
2033,147
2182,176
2183,80


In [48]:
chance <- data.frame(team_att$team_api_id, team_att$chanceCreationPassing, team_att$chanceCreationCrossing, team_att$chanceCreationShooting)
colnames(chance) <- c("team_api_id", "chanceCreationPassing", "chanceCreationCrossing", "chanceCreationShooting")
chance[is.na(chance)] <- 0
# transforms all NA to 0
chance <- mutate(chance, chance_rating = chanceCreationPassing + chanceCreationCrossing + chanceCreationShooting)
# adds all chance attributes together

chance <- data.frame(chance$team_api_id, chance$chance_rating)
colnames(chance) <- c("team_api_id", "chance_rating")
#creates a new data frame having only the team api id and the chance rating

chance <- chance %>% group_by(team_api_id) %>% summarise(chance_rating=max(chance_rating))
# groups similar player ids together and finds the maximum chance rating from the group
chance <- chance %>% arrange(desc(chance_rating))
nrow(chance)
head(chance)

[1] 288

team_api_id,chance_rating
8543,220
8697,216
9875,215
8178,210
8191,210
8344,210


In [49]:
defence <- data.frame(team_att$team_api_id, team_att$defencePressure, team_att$defenceAggression, team_att$defenceTeamWidth)
colnames(defence) <- c("team_api_id", "defencePressure", "defenceAggression", "defenceTeamWidth")
defence[is.na(defence)] <- 0
# transforms all NA to 0
defence <- mutate(defence, defence_rating = defencePressure + defenceAggression + defenceTeamWidth)
# adds all defence attributes together
defence <- data.frame(defence$team_api_id, defence$defence_rating)
colnames(defence) <- c("team_api_id", "defence_rating")
#creates a new data frame having only the team api id and the defence rating

defence <- defence %>% group_by(team_api_id) %>% summarise(defence_rating=max(defence_rating))
# groups similar player ids together and finds the maximum defence rating from the group
defence <- defence %>% arrange(desc(defence_rating))
nrow(defence)
head(defence)

[1] 288

team_api_id,defence_rating
1957,210
7794,210
8021,210
8197,210
8344,210
8350,210


In [52]:
home_goal <- data.frame(Match$home_team_api_id, Match$home_team_goal)
colnames(home_goal) <- c("team_api_id", "goal")
#Data frame having all team pi ids and home goals
away_goal <- data.frame(Match$away_team_api_id, Match$away_team_goal)
colnames(away_goal) <- c("team_api_id", "goal")
#Data frame having all team pi ids and away goals
team_goal <- rbind(home_goal, away_goal)
#appends the two data frame into one
team_goal <- team_goal %>% group_by(team_api_id) %>% summarise(team_goals=sum(goal))
# Groups all similar team api ids together and sums up all the goals in the group
team_goal <- team_goal %>% arrange(desc(team_goals))
nrow(team_goal)
head(team_goal)

[1] 299

team_api_id,team_goals
8634,849
8633,843
9925,695
9823,653
8640,652
8593,647


Now that we have all individual data frames having the attributes that we want in the final table, we join all these data frames together using left join.

In [53]:
Team <- Team_name %>% left_join(buildup1, by = "team_api_id")
Team <- Team %>% left_join(chance, by = "team_api_id")
Team <- Team %>% left_join(defence, by = "team_api_id")
Team <- Team %>% left_join(team_goal, by = "team_api_id")
head(Team)

team_api_id,team_long_name,team_short_name,buildUp_rating,chance_rating,defence_rating,team_goals
9987,KRC Genk,GEN,148,182,205,348
9993,Beerschot AC,BAC,110,185,210,174
10000,SV Zulte-Waregem,ZUL,147,153,200,306
9994,Sporting Lokeren,LOK,159,154,200,279
9984,KSV Cercle Brugge,CEB,145,200,195,213
8635,RSC Anderlecht,AND,149,182,190,427


## Writing the final data frames into their respective csv files

In [54]:
write.csv(Player, "/Users/Amardeep/Documents/Semester_2/Dic/lab_2/player.csv")
write.csv(Team, "/Users/Amardeep/Documents/Semester_2/Dic/lab_2/team.csv")

Writes the final data frames into csv files, please change the url accordingly.